In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-midterm-p3/train.csv
/kaggle/input/2023-ml-midterm-p3/test.csv
/kaggle/input/2023-ml-midterm-p3/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
#데이터 불러오기
train = pd.read_csv("/kaggle/input/2023-ml-midterm-p3/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-midterm-p3/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-midterm-p3/submit_sample.csv")

In [4]:
#데이터 확인하기
train.head()

,index,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,0,8949,103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,1,29725,40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,3,33241,115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
3,4,666,162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
4,5,21651,176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0


In [5]:
#데이터에 필요없는 칼럼 없애주기
train.drop(['index','enrollee_id','city_development_index'],axis=1,inplace=True)
test.drop(['index','enrollee_id','city_development_index'],axis=1,inplace=True)

In [6]:
#독립변수와 종속변수 나눠주기
x=train.drop(['target'],axis=1)
y=train['target'].astype('int64')

In [7]:
#x와 test데이터 합친 data만들어주기
data = pd.concat([x,test])

In [8]:
#dict형태로 바꿔주기
x=x.to_dict("records")
test=test.to_dict("records")
data=data.to_dict("records")

In [9]:
#범주형 데이터 수치형으로 표현하기
from sklearn.feature_extraction import DictVectorizer
dv=DictVectorizer()
data = dv.fit_transform(data)
test = dv.transform(test)
x = dv.transform(x)

In [10]:
# 결측치 채우기
from sklearn.impute import SimpleImputer
si = SimpleImputer()
x = si.fit_transform(x)
test = si.transform(test)

In [11]:
#모델 학습 및 정확도 확인
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model=LogisticRegression(max_iter=1000)
model.fit(x,y)
print(accuracy_score(y,model.predict(x)))

0.7629518465352995


In [12]:
# 제출
submit['target']=model.predict(test)
submit.to_csv("submit.csv",mode='w',header=True,index=False)